IMPORT LIBRARIES

In [2]:
import mysql.connector

In [3]:
import collections
import json 
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import string
import pickle
from surprise import Reader , Dataset
from tabulate import tabulate
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import SVD, Dataset, accuracy
from surprise.model_selection import cross_validate, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder

CONNECT TO MYSQL

In [4]:
# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",  
    user="root",       
    password="Phydy@1311",  
    database="restaurant_rs",  
    charset="utf8mb4",
    port=8000)

cursor = conn.cursor()

In [5]:
# Query data from MySQL
query = "SELECT * FROM reviews;"  
cursor.execute(query)

# Convert data to DataFrame
columns = [desc[0] for desc in cursor.description]  
dataset = cursor.fetchall()
data = pd.DataFrame(dataset, columns=columns)
# Close connection
cursor.close()
conn.close()
# Print the first 5 rows of DataFrame
print(data.head())

   user_id  business_id  rating                date  \
0        1            1       3 2018-07-07 22:09:00   
1        3            3       3 2014-02-05 20:30:00   
2        4            4       5 2015-01-04 00:01:00   
3        5            5       4 2017-01-14 20:54:00   
4        6            6       1 2015-09-23 23:10:00   

                             name  \
0  "Turning Point of North Wales"   
1             "Kettle Restaurant"   
2                         "Zaika"   
3                          "Melt"   
4                      "Dmitri's"   

                                                text  \
0  "If you decide to eat here, just be aware it i...   
1  "Family diner. Had the buffet. Eclectic assort...   
2  "Wow!  Yummy, different,  delicious.   Our fav...   
3  "Cute interior and owner (?) gave us tour of u...   
4  "I am a long term frequent customer of this es...   

                                          attributes  \
0  {"WiFi": "'free'", "HasTV": "False", "Caters":... 

SENTIMENT ANALYSIS

In [6]:
# creating a function that aggregates/combines all the extra reviews made to a particular restaurant into one text

def new_df(data):
    """
    The function takes in a dataframes and groups it by business_id column then combines all the text values in the
    text column into one big text then assigns it to the review column
    
    """
    # drop duplicates based on business_id and reset the index
    df = data.drop_duplicates('business_id').reset_index(drop=True)
    
    # loop through unique business_id values
    for id in data.business_id.unique():
    # extract text for each unique business_id and explode it into separate rows
        text = data.loc[data.business_id == id, 'text'].explode(ignore_index=True)
    # join the exploded text into a single string
        text = ' '.join(text)
        
    # assign the concatenated text to the reviews column for the corresponding business_id
        df.loc[data.business_id == id, 'reviews'] = text
    
    return df

# call the function and create the new df
df = new_df(data)
df.head()  

,user_id,business_id,rating,date,name,text,attributes,categories,hours,location,price,latitude,longitude,bs_rating,review_count,reviews
0,1,1,3,2018-07-07 22:09:00,"""Turning Point of North Wales""","""If you decide to eat here, just be aware it i...","{""WiFi"": ""'free'"", ""HasTV"": ""False"", ""Caters"":...","""Restaurants, Breakfast & Brunch, Food, Juice ...","{""Friday"": ""7:30-15:0"", ""Monday"": ""7:30-15:0"",...","""State:PA, City:North Wales, Address:1460 Beth...",2,40.210200,-75.223600,3.0,169,"""If you decide to eat here, just be aware it i..."
1,3,3,3,2014-02-05 20:30:00,"""Kettle Restaurant""","""Family diner. Had the buffet. Eclectic assort...","{""WiFi"": ""u'free'"", ""HasTV"": ""True"", ""Caters"":...","""Restaurants, Breakfast & Brunch""",{},"""State:AZ, City:Tucson, Address:748 W Starr Pa...",1,32.207200,-110.980900,3.5,47,"""Family diner. Had the buffet. Eclectic assort..."
2,4,4,5,2015-01-04 00:01:00,"""Zaika""","""Wow! Yummy, different, delicious. Our fav...","{""WiFi"": ""u'no'"", ""HasTV"": ""False"", ""Caters"": ...","""Halal, Pakistani, Restaurants, Indian""","{""Friday"": ""11:0-22:0"", ""Sunday"": ""11:0-21:0"",...","""State:PA, City:Philadelphia, Address:2481 Gra...",2,40.079800,-75.025100,4.0,181,"""Wow! Yummy, different, delicious. Our fav..."
3,5,5,4,2017-01-14 20:54:00,"""Melt""","""Cute interior and owner (?) gave us tour of u...","{""WiFi"": ""u'free'"", ""HasTV"": ""True"", ""Music"": ...","""Sandwiches, Beer, Wine & Spirits, Bars, Food,...","{""Friday"": ""11:0-17:0"", ""Monday"": ""0:0-0:0"", ""...","""State:LA, City:New Orleans, Address:2549 Bank...",2,29.962100,-90.088000,4.0,32,"""Cute interior and owner (?) gave us tour of u..."
4,6,6,1,2015-09-23 23:10:00,"""Dmitri's""","""I am a long term frequent customer of this es...","{""WiFi"": ""u'no'"", ""HasTV"": ""False"", ""Caters"": ...","""Mediterranean, Restaurants, Seafood, Greek""","{""Friday"": ""17:30-22:0"", ""Sunday"": ""17:0-21:0""...","""State:PA, City:Philadelphia, Address:795 S 3r...",2,39.938000,-75.148100,4.0,273,"""I am a long term frequent customer of this es..."


In [7]:
# decompressing the attributes column into  new 'attributes_true' column
def decompress(x):
    """
    The function takes in a dictionary and returns only the keys that have their values not being False   
    """
      
    list_ = []
# evaluate the attributes column to convert it from a string to a dictionary
    data_dict = eval(x)
    
# iterate through the key-value pairs in the dictionary
    for key, val in data_dict.items():
# check if the key is in the specified categories and if the value is not "None"
        if (key in ['Ambience', 'GoodForMeal', 'BusinessParking']) and (val != "None"):
# if conditions are met, further iterate through sub-dictionary
            for key_, val_ in eval(data_dict[key]).items():
# if the sub-dictionary value is true, append it to the list
                if val_:
                    list_.append(f'{key}_{key_}')
        else:
# if the value is not false, append the key to the list
            if val != 'False':
                list_.append(key)
    
# join the list of selected attribute names into a space-separated string
    return " ".join(list_)

# create a new column 'attributes_true' in the df by applying the decompress function
# include a condition to handle cases where attributes is 'Not-Available'
df['attributes_true'] = df.attributes.apply(lambda x: decompress(x) if x != 'Not-Available' else ' ')

In [8]:
# confirming if the newly created column has performed as expected
print("Before:")
print(eval(df.attributes[0]))
print('\n After:')
df['attributes_true'][0]      # Print the result for the first row of 'attributes'

Before:
{'WiFi': "'free'", 'HasTV': 'False', 'Caters': 'False', 'Alcohol': "'none'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}", 'HappyHour': 'False', 'NoiseLevel': "u'average'", 'BikeParking': 'False', 'DogsAllowed': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'OutdoorSeating': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'RestaurantsAttire': "'casual'", 'RestaurantsTakeOut': 'True', 'RestaurantsDelivery': 'True', 'WheelchairAccessible': 'True', 'BusinessAcceptsBitcoin': 'False', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'True', 'RestaurantsGoodForGroups': 'True', 'BusinessAcceptsCreditCards': 'True'}

 After:


'WiFi Alcohol Ambience_casual NoiseLevel GoodForKids GoodForMeal_lunch GoodForMeal_brunch GoodForMeal_breakfast OutdoorSeating BusinessParking_lot RestaurantsAttire RestaurantsTakeOut RestaurantsDelivery WheelchairAccessible RestaurantsPriceRange2 RestaurantsTableService RestaurantsGoodForGroups BusinessAcceptsCreditCards'

In [9]:
# merging different columns to form one column of text 
df['details']=df[['attributes_true','categories','reviews']].apply(lambda x: ''.join(x), axis=1)

# previewing the first row value in the new column
df.details[0]

'WiFi Alcohol Ambience_casual NoiseLevel GoodForKids GoodForMeal_lunch GoodForMeal_brunch GoodForMeal_breakfast OutdoorSeating BusinessParking_lot RestaurantsAttire RestaurantsTakeOut RestaurantsDelivery WheelchairAccessible RestaurantsPriceRange2 RestaurantsTableService RestaurantsGoodForGroups BusinessAcceptsCreditCards"Restaurants, Breakfast & Brunch, Food, Juice Bars & Smoothies, American (New), Coffee & Tea, Sandwiches""If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it\'s other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker." "This is the second time we tried turning point at th

In [10]:
# dropping columns
df.drop(columns=['attributes_true','reviews'], inplace=True)

 The column text contains many symbols, punctuations and stopwords, next we shall remove the symbols and tokenize the column into a bag of words. These reasons serve to prepare text data for various text analysis and NLP tasks. It tokenizes the text, applies stemming, and standardizes the text for downstream processing, making it easier to analyze and extract meaningful information from the text.



In [11]:
# first create a pattern that strips all the non-word characters from words during tokenization
pattern =r"(?u)\b\w\w+\b"

# instantiate the tokenizer
tokenizer = RegexpTokenizer(pattern)

# instantiating the stemmer
stemmer = SnowballStemmer(language="english")

# creating a function to tokenize and stem words
def stem_and_tokenize(list_):
    tokens = tokenizer.tokenize(list_)
    return [stemmer.stem(token) for token in tokens]

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
# instantiating the stop words
stopwords=stopwords.words('english')
# stemming the stopwords for uniformity while removing stopwords
stopwords=[ stemmer.stem(i) for i in stopwords]


tfidf = TfidfVectorizer( max_features=200 , 
                        stop_words=stopwords,
                        tokenizer= stem_and_tokenize
#                         ngram_range=(1, 2), 
#                         min_df=0, 
                        )


# fitting and transforming the details column to extract the top 200 features
tfidf_matrix=tfidf.fit_transform(df['details'])

# previewing the tfidf matrix
pd.DataFrame.sparse.from_spmatrix(tfidf_matrix, columns=tfidf.get_feature_names_out()).head()

c:\Users\OS\VALY_FinalProject\.venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,also,alway,amaz,anoth,appet,area,around,ask,atmospher,attent,...,walk,want,way,well,went,wine,wonder,work,worth,would
0,0.061112,0.015834,0.016384,0.121125,0,0.033286,0.016721,0.081323,0.018055,0.058282,...,0.017611,0.015551,0.065457,0.093172,0.04839,0,0,0.034368,0.01758,0.045063
1,0.080986,0,0,0,0,0.029407,0,0.057477,0.031901,0.102981,...,0.031117,0,0.057829,0.054876,0.028501,0,0,0.18218,0,0.132704
2,0.048283,0.05004,0.051779,0.021874,0.013755,0.063117,0.031706,0.01028,0.045647,0.036838,...,0,0.058976,0.010343,0.07852,0.081561,0.015968,0.02385,0.043446,0.033335,0.085447
3,0.033481,0,0,0,0.047692,0.072946,0.036644,0,0.039566,0,...,0,0,0.035862,0.068061,0.035348,0.276817,0,0,0,0.098753
4,0.054822,0.186682,0.075589,0.02661,0.055778,0.025594,0.034285,0.025012,0.009255,0.009959,...,0.018055,0.031887,0.041942,0.02388,0.082684,0.168351,0.058029,0.008809,0.045059,0.076998


In [65]:
# creating a matrix of the cosine similarities of the various rows based on the tidf scores
cosine_similarity=cosine_similarity(tfidf_matrix)
print("shape: ",cosine_similarity.shape)

# viewing the first column
cosine_similarity[0]

shape:  (4132, 4132)


array([1.        , 0.66526074, 0.59666654, ..., 0.53302993, 0.53687354,
       0.49664839])

In [66]:
import os

In [67]:
os.makedirs('./data', exist_ok=True)

In [68]:
pickle.dump(tfidf_matrix, open('./data/tfidf_matrix.pkl', 'wb'))

In [69]:
# saving our data for deployment
pickle.dump(cosine_similarity, open('./data/cosine_similarity.pkl', 'wb'))
pickle.dump(df, open('./data/data.pkl', 'wb'))
print("Files saved...")

Files saved...


In [70]:
with open('./data/cosine_similarity.pkl', 'rb') as file:
    cosine_similarity=pickle.load(file)
    
with open('./data/data.pkl', 'rb') as file:
    df=pickle.load(file)

CONTENT-BASED MODEL

In [72]:
import folium

In [71]:
# creating a folium_map function that displays restaurant lovations
def folium_map(data):
    """
    The function takes in a dataframe and using the latitude and longitude columns displays a map showing the locations of 
    all the restaurants available in the input data
    """
    # reseting the index in the input dataframe
    dff=data.reset_index(drop=True)

# Set up center latitude and longitude
    center_lat = dff['latitude'][0]
    center_long = dff['longitude'][0]

# Initialize map with center lat and long
    map_ =folium.Map([center_lat,center_long], zoom_start=7)

# Adjust this limit to see more or fewer businesses
    limit=dff.shape[0]
    print(f"{limit-1} Restaurant Locations")
    for index in range(limit-1):
        # Extract information about business
        lat = dff.loc[index,'latitude']
        long = dff.loc[index,'longitude']
        name = dff.loc[index,'name']
        rating = dff.loc[index,'bs_rating']
        location = dff.loc[index,'location']
        details = "{}\nStars: {} {}".format(name,rating,location)

# Create popup with relevant details
        popup = folium.Popup(details,parse_html=True)

# Create marker with relevant lat/long and popup
        marker = folium.Marker(location=[lat,long], popup=popup)

        marker.add_to(map_)

    return display(map_)  # returning a map display

In [ ]:
folium_map(data=df.loc[:10])

In [74]:
def content_based(df=df, name:str= None , rating:int =1, num:int=5, text: str=None, location:str = None):
    """
    The function takes the following input;
    
    df: DataFrame - a dataframe containing unique resturants
    name: str - name of restaurant to recommend similar restaurants
    num:int - number of restaurants to recommend
    location: string - preferred location
    rating: string - preferred rating of restaurant
    text: - User preferences inform of text
    
    Then based on the input parameters offers similar restaurants according to the input parameters to users
    """
    
    if name:
        index_=df.loc[df.name== name].index[0]                          # find the index of the input name
        sim=list(enumerate(cosine_similarity[index_]))                  # extract similarity vector of that name index
        sim=sorted(sim, key=lambda x: x[1], reverse=True)[1:num+1]      # arrange the vector values in ascending order
        indices= [i[0] for i in sim]                                    # Extract the indices of the top high scores
        print(f"Top {num} Restaurants Like [{name}]")
        
        # if the location parameter is passed then the dataframe is filtered based on the input location
        if location:                                                
            df=df.loc[ (df['bs_rating']>=rating) & ( df.location.str.contains(location))]
            folium_map(df)
        else: 
            df= df.loc[ (df['bs_rating']>=rating) ] 
        # filtering the data based on the selected indices    
        df=df.loc[indices,('name','bs_rating','review_count','location')].sort_values('bs_rating', ascending=False)
        return  df.reset_index(drop=True)
    
    # if the name is None then switch to other parameters
    else:
        # if the text has a passed input values then this if statement runs            
        if text: 
                text=text.lower()                                           # converting the text into lowercase
                tokens=stem_and_tokenize(text)                              # tokenizing and stemming the words
                tokens=[ word for word in tokens if word not in stopwords]  # removing stopwords
                text_set=set(tokens)                                        # taking only unique words
                
                if location: # using entered location to filter the data
                    df=df.loc[ (df.location.str.contains(location)) & (df['bs_rating']>=rating)].reset_index(drop=True)

                vectors=[] # creating an emplty list to append the intersection values
                for words in df.details:                                     # looping over the text in the details column
                    words=words.lower()                                      # lowering the text
                    words=stem_and_tokenize(text)                            # tokenizing and stemming the words
                    words=[ word for word in tokens if word not in stopwords] # removing stopwords
                    words=set(words)                                         # taking only unique words
                    vector=text_set.intersection(words)                      # checking for intersection with entered text 
                    vectors.append(len(vector))                              # appending value to vectors list
                    
                vectors=sorted(list(enumerate(vectors)), key= lambda x: x[1], reverse=True)[:num] # sorting the list in desc
                indices= [i[0] for i in vectors]                                         # selecting indices of top values
                print(f"Top {num} Best Restaurants Based on entered text:")
                # using the indices fileter the dataframe 
                df=df.loc[indices].sort_values(by=['bs_rating','review_count'],ascending=False)
                if location: folium_map(df)                                   # calling the folim_map of the selected values
                return df[['name','bs_rating','review_count','location']].reset_index(drop=True) # offering recommendations
        
        # the if only location is entered as a parameter then the top businesses in that location are recommended
        if location:
            df=df.loc[ df.location.str.contains(location)& (df['bs_rating']>=rating)] #filtering dataframe
            df=df.sort_values(['review_count','bs_rating'])[:num]     # sorting in descending order
            folium_map(data=df)
            return df[['name','bs_rating','review_count','location']].reset_index(drop=True) # offering recommendations
         
        # if both the name, text and location are None the most popular restaurants are recommended
        else:                
            df=df.loc[data['bs_rating']>=rating].sort_values(by=['review_count','bs_rating'],ascending=False)[:num]
            if location: folium_map(data=df)
            print("Most Popular Restaurants")
            return df[['name','bs_rating','review_count','location']].reset_index(drop=True)
    

In [75]:
# running the recommender on default parameters
content_based()

Most Popular Restaurants


,name,bs_rating,review_count,location
0,"""Luke""",4.0,4554,"""State:LA, City:New Orleans, Address:333 Saint..."
1,"""Santa Barbara Shellfish Company""",4.0,2404,"""State:CA, City:Santa Barbara, Address:230 Ste..."
2,"""Prep & Pastry""",4.5,2126,"""State:AZ, City:Tucson, Address:2660 N Campbel..."
3,"""Surrey's Caf\u00c3\u00a9 & Juice Bar""",4.5,2084,"""State:LA, City:New Orleans, Address:1418 Maga..."
4,"""Mr. B's Bistro""",4.0,2064,"""State:LA, City:New Orleans, Address:201 Royal..."


COLLABORATIVE FILTERING MODEL

MODEL-BASED

In [96]:
 #selecting specific columns that are relevant for collaborative filtering models
new_df = data[['user_id', 'business_id', 'rating']]

# using Reader() from surprise module to convert dataframe into surprise dataformat
# instantiating a readerobject
reader = Reader()

# using the reader to read the trainset
data_2 = Dataset.load_from_df(new_df,reader)

dataset = data_2.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of Restaurants: ', dataset.n_items)

Number of users:  59366 

Number of Restaurants:  4132


In [ ]:
# Set seed for reproducibility
np.random.seed(42)
# Initialize model with random_state
svd = SVD(random_state=42)
# Run cross-validation with a single processing thread to ensure consistent results
results = cross_validate(svd, data_2, cv=5, n_jobs=1)
# Print results
for values in results.items():
    print(values)
print("-------------------------")
print("Mean RMSE: ", results['test_rmse'].mean())
print("Mean MAE: ", results['test_mae'].mean())

('test_rmse', array([1.23644536, 1.23958288, 1.24056992, 1.23267019, 1.2380284 ]))
('test_mae', array([0.99234466, 0.99366821, 0.99557203, 0.99134174, 0.99388224]))
('fit_time', (1.4077303409576416, 1.0587482452392578, 1.01417875289917, 1.0790128707885742, 1.0114905834197998))
('test_time', (0.07374954223632812, 0.07540035247802734, 0.07733678817749023, 0.07379460334777832, 0.0700829029083252))
-------------------------
Mean RMSE:  1.2374593507550091
Mean MAE:  0.9933617753230081


In [ ]:
# Set seed
np.random.seed(42)
# Define a dictionary params with hyperparameter values to be tested
param_grid = {'n_factors': [20, 50, 100], # number of factors for matrix factorization
         'reg_all': [0.02, 0.05, 0.1] , 'random_state': [42]} # regularization term
# create a GridSearchCV object 'g_s_svd' for hyperparameter tuning
g_s_svd = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1)
# Fit the GridSearchCV object to the data to find the best hyperparameters
g_s_svd.fit(data_2)
# Print results
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 1.2359924487446718, 'mae': 0.9915505731155146}
{'rmse': {'n_factors': 20, 'reg_all': 0.05, 'random_state': 42}, 'mae': {'n_factors': 20, 'reg_all': 0.02, 'random_state': 42}}


In [133]:
# created an instance of the SVD model with specified hyperparameters
svd = SVD(n_factors= 20, reg_all=0.02)
# fit the SVD model to the dataset
svd.fit(dataset)

In [ ]:
# using the model, we'll try and make a rating prediction of user 15, on restaurant with id "Pns2l4eNsfO8kk83dixA6A"
svd.predict("15", "Pns2l4eNsfO8kk83dixA6A")

Prediction(uid='15', iid='Pns2l4eNsfO8kk83dixA6A', r_ui=None, est=3.8108396650213523, details={'was_impossible': False})

In [ ]:
# define a function named 'restaurant_rater' that takes user inputs to rate restaurants
def restaurant_rater(data=df,num:int=3, location:str =None, category:str =None):
    
    """
    The functions takes the following inputs:
    data: DataFrame - a dataframe containing only rows of the unique business 
    num: int - number of ratings
    location: string - preferred location
    category: string - preferred category of restaurant
    
    Then randomly draws restaurant names from the dataframe for the user to rate
    """
    df_restaurant=data
    #assigning the rating user a user_id
    user_id=df_restaurant.user_id.max()+1                                               
    rating_df = pd.DataFrame()   # create an empty dataframe to store user rated restaurants
    # continue the loop until the desired num is collected
    while num > 0:
        # select a random restaurant that matches the specified location
        if location: 
            restaurant = df_restaurant[df_restaurant['location'].str.contains(location)].sample(1)
         # select a random restaurant that matches the specified category    
        elif category:
            restaurant = df_restaurant[df_restaurant['categories'].str.contains(category)].sample(1)
        else:  # or else selects a random restaurant
            restaurant = df_restaurant.sample(1)
        # prints the selected restaurant    
        print(tabulate(restaurant[['name','bs_rating','categories']], headers='keys', tablefmt='fancy_grid', showindex=False))
        # asks for rating from user
        rating = input("How do you rate this restaurant on a scale of 1-5, Enter: ")
        
        # creating a function that checks the validity of the entered rating ie should be bewteen 1-5
        def checker(rating):
            if (len(rating)!= 0):
                while (float(rating)>5) :
                    print("Enter valid rating, scale of 1-5 or Enter")  
                    rating= input()
                return rating
            else: return rating
            
        # calling the function to confirm the selected rating 
        rating = checker(rating)
        if len(rating) == 0:                                        # if no rating is entered 
            num-=1                                                  # the jumps to select another restaurant
            continue
        else:
            restaurant.loc[:,('user_id')]= user_id                   # then the selected restaurant is assigned the user id
            restaurant.loc[:,('rating')]= rating
            rating_df=pd.concat([rating_df,restaurant], axis=0)   # the movie is added to our new user rated dataframe
            num-=1                                                  # then another restaurant is suggested till num==0
            # return the list of user ratings and restaurant information
    return rating_df

In [104]:
# rating 4  restaurants that have sandwiches on their menu
restaurant_rater( num=4, category='Sandwiches')

╒══════════════════════════╤═════════════╤═══════════════════════════════════════════╕
│ name                     │   bs_rating │ categories                                │
╞══════════════════════════╪═════════════╪═══════════════════════════════════════════╡
│ "Johny's Original Pizza" │         3.5 │ "Italian, Sandwiches, Pizza, Restaurants" │
╘══════════════════════════╧═════════════╧═══════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒═════════════════════════╤═════════════╤══════════════════════════════════════════════════════════════════════════╕
│ name                    │   bs_rating │ categories                                                               │
╞═════════════════════════╪═════════════╪══════════════════════════════════════════════════════════════════════════╡
│ "Stacie's Cottage Cafe" │           4 │ "Coffee & Tea, Cafes, Breakfast & Brunch, Restaurants, Sandwiches, Food" │
╘═════════════════════════╧═════════════╧══════════════════════════════════════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒══════════╤═════════════╤══════════════════════════════════════╕
│ name     │   bs_rating │ categories                           │
╞══════════╪═════════════╪══════════════════════════════════════╡
│ "Arby's" │         1.5 │ "Sandwiches, Fast Food, Restaurants" │
╘══════════╧═════════════╧══════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒════════════════════════════╤═════════════╤══════════════════════════════════════════════════════════════════════════════════════════════╕
│ name                       │   bs_rating │ categories                                                                                   │
╞════════════════════════════╪═════════════╪══════════════════════════════════════════════════════════════════════════════════════════════╡
│ "Joe Bob's Chicken Palace" │         3.5 │ "Chicken Wings, American (Traditional), Restaurants, Shopping, Sandwiches, Shopping Centers" │
╘════════════════════════════╧═════════════╧══════════════════════════════════════════════════════════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


,user_id,business_id,rating,date,name,text,attributes,categories,hours,location,price,latitude,longitude,bs_rating,review_count,details
1591,79132,2639,4,2017-01-28 00:12:00,"""Johny's Original Pizza""","""Ordered for pick up and was told 45 minutes. ...","{""WiFi"": ""'no'"", ""HasTV"": ""True"", ""Caters"": ""T...","""Italian, Sandwiches, Pizza, Restaurants""","{""Friday"": ""11:0-22:0"", ""Monday"": ""11:0-22:0"",...","""State:PA, City:Prospect Park, Address:547 Che...",1,39.886900,-75.301700,3.5,54,WiFi HasTV Caters Alcohol NoiseLevel BikeParki...
3129,79132,6241,4,2015-10-18 15:25:00,"""Stacie's Cottage Cafe""","""Great little cafe with quality food, and frie...","{""WiFi"": ""u'free'"", ""HasTV"": ""True"", ""Ambience...","""Coffee & Tea, Cafes, Breakfast & Brunch, Rest...","{""Friday"": ""7:30-14:30"", ""Monday"": ""7:30-14:30...","""State:FL, City:Saint Petersburg, Address:501 ...",1,27.772500,-82.640500,4.0,27,WiFi HasTV NoiseLevel BikeParking GoodForKids ...
3764,79132,8430,4,2018-02-24 03:44:00,"""Arby's""","""Stopped for a meal. First and last visit. Hug...","{""WiFi"": ""'no'"", ""HasTV"": ""True"", ""Caters"": ""F...","""Sandwiches, Fast Food, Restaurants""",{},"""State:IL, City:Godfrey, Address:5509 Godfrey ...",1,38.946400,-90.190300,1.5,6,WiFi HasTV Alcohol DriveThru NoiseLevel GoodFo...
570,79132,849,4,2014-03-15 18:07:00,"""Joe Bob's Chicken Palace""","""Well, well.... Where do I start with this pla...","{""WiFi"": ""'free'"", ""HasTV"": ""True"", ""Caters"": ...","""Chicken Wings, American (Traditional), Restau...","{""Friday"": ""11:0-23:0"", ""Monday"": ""11:0-21:0"",...","""State:NV, City:Reno, Address:7689 S Virginia ...",2,39.456400,-119.781800,3.5,62,WiFi HasTV Caters Alcohol Ambience_divey Happy...


In [86]:
restaurant_rater(location='PA')

╒════════════════╤═════════════╤═══════════════════════════════════════════╕
│ name           │   bs_rating │ categories                                │
╞════════════════╪═════════════╪═══════════════════════════════════════════╡
│ "Brindle Cafe" │           3 │ "Sandwiches, Restaurants, American (New)" │
╘════════════════╧═════════════╧═══════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒════════════════════╤═════════════╤═══════════════════════════════════╕
│ name               │   bs_rating │ categories                        │
╞════════════════════╪═════════════╪═══════════════════════════════════╡
│ "Mike's Breakfast" │           3 │ "Restaurants, Breakfast & Brunch" │
╘════════════════════╧═════════════╧═══════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒═════════════╤═════════════╤═══════════════╕
│ name        │   bs_rating │ categories    │
╞═════════════╪═════════════╪═══════════════╡
│ "Twenty 21" │           3 │ "Restaurants" │
╘═════════════╧═════════════╧═══════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


,user_id,business_id,rating,date,name,text,attributes,categories,hours,location,price,latitude,longitude,bs_rating,review_count,details
2456,79132,4447,4,2012-06-28 04:19:00,"""Brindle Cafe""","""Is this place even open anymore? I live liter...","{""WiFi"": ""u'free'"", ""HasTV"": ""True"", ""Caters"":...","""Sandwiches, Restaurants, American (New)""","{""Friday"": ""11:0-21:0"", ""Tuesday"": ""11:0-21:0""...","""State:PA, City:Philadelphia, Address:221 E Gi...",1,39.969400,-75.133200,3.0,9,WiFi HasTV Caters Alcohol NoiseLevel GoodForKi...
3827,79132,8682,4,2015-03-06 16:58:00,"""Mike's Breakfast""","""I ordered delivery from mikes and the minimum...","{""WiFi"": ""u'no'"", ""HasTV"": ""True"", ""Caters"": ""...","""Restaurants, Breakfast & Brunch""",{},"""State:PA, City:Philadelphia, Address:6201 N F...",1,40.045700,-75.118900,3.0,15,WiFi HasTV Alcohol Ambience_casual NoiseLevel ...
3314,79132,6773,4,2007-06-08 18:02:00,"""Twenty 21""","""Came here during weekday lunch w/ my cousin.\...","{""Alcohol"": ""u'full_bar'"", ""GoodForKids"": ""Fal...","""Restaurants""",{},"""State:PA, City:Philadelphia, Address:2005 Mar...",3,39.954100,-75.174700,3.0,8,Alcohol OutdoorSeating BusinessParking_street ...


In [ ]:
def cf_model(df=df,num:int=3, location:str=None, name=None , text=None):
    """
    The function takes the following inputs;
    
    df: DataFrame - a dataframe containing unique restaurants
    name: str - name of restaurant to recomend similar restaurants
    num:int - number of restaurants to recommend
    location: string - preferred location
    category: string - preferred category of restaurant
    text: - User preferences in form of text
    
    The function then takes user ratings and appends them to the dataframe then fits this new dataframe to the SVD model
    Then predicts this user rating on all the restaurants in the dataframe then selects the top rating predictions and 
    recommends those restaurants
    """
    
    # calling the rater function for user to enter restaurant ratings
    user_ratings=restaurant_rater(num=num, location=location)
    
    # when the user ratings come back blank, ie no ratings given ie cold start problem
    # then the content-based method is called, which then makes the recommendations
    if len(user_ratings)==0:
        #return content_based(df=df,num=num,name=name,location=location,text=text,)
        pass
    # then add the user ratings to our df
    df=pd.concat([df,user_ratings],axis=0)

    # convert the new dataset into surprise format
    dataset = Dataset.load_from_df(df[['user_id','business_id','rating']],reader)
    
    # then fit the surprise data to the SVD
    svd = SVD(n_factors= 20, reg_all=0.02)
    svd.fit(dataset.build_full_trainset())
    
    # extract the user rating in the ratings dataframe
    user_id=user_ratings['user_id'].values[0]
    
    # select a random restaurant that matches the specified category
    if location: 
            df = df.loc[df['location'].str.contains(location)]
        
    #create an empty list to append the model predictions
    user_predictions=[]
    
    # loopin over all the unique restaurant ids in the dataframe and appending the predictions to user prediction list
    for  iid in df.business_id.unique():
        user_predictions.append( (iid , svd.predict(user_id, iid)[3]))
    
    # sorting the predictions in descending order of the predictions values
    top_pred = sorted(user_predictions , key =lambda x: x[1], reverse=True)
    
    # selecting the top 'num'(number of predictions) prediction indicies
    indices=[i[0] for i in top_pred[:num]]  
    
    #using the extracted indices extract the restaurant titles
    rec = df.loc[ df['business_id'].isin(indices)].sort_values('bs_rating', ascending=False)
    display(folium_map(rec))
    
    #then retun the resturants details to user
    return rec[['name','bs_rating','location']].reset_index(drop=True)  

In [147]:
# based on default parameters
cf_model()

╒═════════════════╤═════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ name            │   bs_rating │ categories                                                                                                              │
╞═════════════════╪═════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ "El Gallo Giro" │         3.5 │ "Beer, Wine & Spirits, Nightlife, Bars, New Mexican Cuisine, Food, Tacos, Restaurants, Cocktail Bars, Mexican, Seafood" │
╘═════════════════╧═════════════╧═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒═══════════╤═════════════╤═════════════════════════════════════════════╕
│ name      │   bs_rating │ categories                                  │
╞═══════════╪═════════════╪═════════════════════════════════════════════╡
│ "Quiznos" │           3 │ "Restaurants, Fast Food, Delis, Sandwiches" │
╘═══════════╧═════════════╧═════════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒═════════╤═════════════╤═════════════════════════════════════════════════════╕
│ name    │   bs_rating │ categories                                          │
╞═════════╪═════════════╪═════════════════════════════════════════════════════╡
│ "Brodo" │           3 │ "Coffee & Tea, Restaurants, Sandwiches, Food, Soup" │
╘═════════╧═════════════╧═════════════════════════════════════════════════════╛
2 Restaurant Locations


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


None

,name,bs_rating,location
0,"""Jimmy John's""",4.5,"""State:IN, City:Indianapolis, Address:10110 E ..."
1,"""Jack Patrick's Bar & Grill""",4.0,"""State:MO, City:Saint Louis, Address:1000 Oliv..."
2,"""Castrillo's Pizza of Donelson""",3.0,"""State:TN, City:Nashville, Address:2543 Lebano..."


In [92]:
# offering recommendations based on a specified location
cf_model(num=5,location='LA')

╒═══════════════════════════════════════════╤═════════════╤════════════════════════════════════════════╕
│ name                                      │   bs_rating │ categories                                 │
╞═══════════════════════════════════════════╪═════════════╪════════════════════════════════════════════╡
│ "Stella Maris Restaurant & Hookah Lounge" │         4.5 │ "Food, Cafes, Middle Eastern, Restaurants" │
╘═══════════════════════════════════════════╧═════════════╧════════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒═════════════════╤═════════════╤═══════════════════════════════════════════════════════════════════╕
│ name            │   bs_rating │ categories                                                        │
╞═════════════════╪═════════════╪═══════════════════════════════════════════════════════════════════╡
│ "The Mason Jar" │         3.5 │ "Sandwiches, American (New), American (Traditional), Restaurants" │
╘═════════════════╧═════════════╧═══════════════════════════════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒════════════════════════════╤═════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ name                       │   bs_rating │ categories                                                                                                          │
╞════════════════════════════╪═════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ "Comp\u00c3\u00a8re Lapin" │         4.5 │ "Personal Chefs, Event Planning & Services, Restaurants, Caribbean, American (New), American (Traditional), French" │
╘════════════════════════════╧═════════════╧═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╛


...

C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒═══════════════════════╤═════════════╤═════════════════════════════════════════════╕
│ name                  │   bs_rating │ categories                                  │
╞═══════════════════════╪═════════════╪═════════════════════════════════════════════╡
│ "Ralph's on the Park" │           4 │ "American (New), Restaurants, Cajun/Creole" │
╘═══════════════════════╧═════════════╧═════════════════════════════════════════════╛
4 Restaurant Locations


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


None

,name,bs_rating,location
0,"""We've Got Soul""",5.0,"""State:LA, City:New Orleans, Address:8140 Will..."
1,"""Frady's One Stop Food Store""",4.5,"""State:LA, City:New Orleans, Address:3231 Daup..."
2,"""Blue Plate Cafe""",4.0,"""State:LA, City:New Orleans, Address:1330 Pryt..."
3,"""The Howlin' Wolf""",3.5,"""State:LA, City:New Orleans, Address:907 S Pet..."
4,"""Sassafras Restaurant""",3.5,"""State:LA, City:New Orleans, Address:2501 Leon..."


In [93]:
cf_model(location='New Orleans',text="restaurant with delicious crabs and nice outdoor setting")

╒══════════════════╤═════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ name             │   bs_rating │ categories                                                                                                                   │
╞══════════════════╪═════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ "Cafe Reconcile" │         4.5 │ "Cajun/Creole, Local Services, Southern, Soul Food, Food, Restaurants, Community Service/Non-Profit, Food Delivery Services" │
╘══════════════════╧═════════════╧══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒════════════╤═════════════╤════════════════════════════════════════╕
│ name       │   bs_rating │ categories                             │
╞════════════╪═════════════╪════════════════════════════════════════╡
│ "Santa Fe" │         3.5 │ "Mexican, Restaurants, Latin American" │
╘════════════╧═════════════╧════════════════════════════════════════╛


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


╒════════════════╤═════════════╤════════════════════════════════════════════╕
│ name           │   bs_rating │ categories                                 │
╞════════════════╪═════════════╪════════════════════════════════════════════╡
│ "Taco Del Mar" │           2 │ "Restaurants, Mexican, Fast Food, Tex-Mex" │
╘════════════════╧═════════════╧════════════════════════════════════════════╛
2 Restaurant Locations


C:\Users\OS\AppData\Local\Temp\ipykernel_15448\2056402799.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  restaurant.loc[:,('rating')]= rating


None

,name,bs_rating,location
0,"""Chappy's Restaurant""",4.5,"""State:LA, City:New Orleans, Address:6106 Maga..."
1,"""Blue Plate Cafe""",4.0,"""State:LA, City:New Orleans, Address:1330 Pryt..."
2,"""Taco Del Mar""",3.5,"""State:LA, City:New Orleans, Address:514 City ..."
